In [1]:
import geopandas as gpd
import geemap
import pandas as pd
import numpy as np
import ee
import re
import plotnine as p9
import sys
from datetime import timedelta
import os
from datetime import datetime

In [2]:
ee.Initialize()

In [15]:

shp_name = "SquareBuffer1km"
shp_dir = "UbonRatchathani"
sq1km = gpd.read_file(os.path.join(shp_dir, shp_name + ".shp"))
sq1km_ee = geemap.geopandas_to_ee(sq1km)

In [18]:
google_drive_path = '/Users/gopal/Google Drive/_Research/Research projects/ML/downloadsentinel'
shp_gdrive_path = os.path.join(google_drive_path,shp_name)
shp_gee_folder_name = shp_name + '_GEE'
shp_gee_folder_path = os.path.join(shp_gdrive_path, shp_gee_folder_name)

if not os.path.exists(shp_gdrive_path):
    os.mkdir(shp_gdrive_path)
if not os.path.exists(shp_gee_folder_path):
    os.mkdir(shp_gee_folder_path)

In [156]:
Map = geemap.Map()

Map.addLayer(sq1km_ee,{},'sq1km_ee')
Map.centerObject(sq1km_ee, 13)

In [20]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
  .filterBounds(sq1km_ee) #\
  # .filterDate("2020-01-01","2020-02-01")
# s1.size().getInfo()
s1_image_names = s1.aggregate_array("system:index").getInfo()

In [60]:
s1_table = pd.DataFrame({'names' : s1_image_names})
s1_table['datestr'] = [re.sub(".*_.*_.*_.*_([0-9]+)T[0-9]+_.*","\\1",x) for x in s1_table["names"]]
s1_table['date'] = pd.to_datetime(s1_table['datestr'], format = "%Y%m%d")
s1_table = s1_table.sort_values('date')

# s1_table['day_diff'] = 1
# print(np.array(s1_table.loc[1:2,'date']))
# print(np.array(s1_table.loc[0:1,'date']))
# print(s1_table['date'].shift(0))
s1_table['days_until_next'] = (s1_table['date'].shift(-1) - s1_table['date']).dt.days

# np.array(s1_table.loc[1:2,'date']) - np.array(s1_table.loc[0:1,'date'])
s1_table['doy'] = [int(datetime.strftime(x, '%j')) for x in s1_table['date']]
s1_table['year'] = [int(datetime.strftime(x, '%Y')) for x in s1_table['date']]
s1_table
# s1_tablep

,names,datestr,date,days_until_next,doy,year
0,S1A_IW_GRDH_1SDV_20150304T225147_20150304T2252...,20150304,2015-03-04,24.0,63,2015
1,S1A_IW_GRDH_1SDV_20150328T225148_20150328T2252...,20150328,2015-03-28,24.0,87,2015
2,S1A_IW_GRDH_1SDV_20150421T225148_20150421T2252...,20150421,2015-04-21,48.0,111,2015
3,S1A_IW_GRDH_1SDV_20150608T225154_20150608T2252...,20150608,2015-06-08,24.0,159,2015
4,S1A_IW_GRDH_1SDV_20150702T225157_20150702T2252...,20150702,2015-07-02,48.0,183,2015
...,...,...,...,...,...,...
392,S1A_IW_GRDH_1SDV_20221216T225252_20221216T2253...,20221216,2022-12-16,8.0,350,2022
393,S1A_IW_GRDH_1SDV_20221224T111302_20221224T1113...,20221224,2022-12-24,4.0,358,2022
394,S1A_IW_GRDH_1SDV_20221228T225251_20221228T2253...,20221228,2022-12-28,8.0,362,2022
395,S1A_IW_GRDH_1SDV_20230105T111301_20230105T1113...,20230105,2023-01-05,4.0,5,2023


In [62]:
doy_select = np.arange(1, 370, 6)


def resample_nearest_days(doy_select, days_orig):
    """
    This function returns an np array, days_nearest, which reflects the unique
    values of days_orig that are closest to each value of doy_select
    doy_select: np.array
    days_orig: np.array
    """
    doy_select_mat = np.expand_dims(days_orig, 0).repeat(len(doy_select), 0)
    days_orig_mat = np.expand_dims(doy_select, 1).repeat(len(days_orig), 1)
    doy_nearest_idx = np.argmin(np.abs(days_orig_mat - doy_select_mat),axis = 1)
    days_nearest = days_orig[np.unique(doy_nearest_idx)]
    return days_nearest


# get names in each year
resample_names = []
for year in pd.unique(s1_table['year']):
    # year = 2016
    days_orig = np.array(s1_table[s1_table['year'] == year]['doy'])
    days_resampled = resample_nearest_days(doy_select, days_orig)
    # days_resampled.shape
    s1_im_year = s1_table[s1_table['year'] == year]
    # s1_im_year = s1_im_year[~s1_im_year['doy'].duplicated()]
    s1_im_year_resampled_names = s1_im_year.names[
        (s1_table['doy'].isin(days_resampled)) &
        (~s1_im_year['doy'].duplicated())]
    resample_names = resample_names + s1_im_year_resampled_names.values.tolist()


s1_table['sample'] = s1_table['names'].isin(resample_names)
s1_table['downloaded'] = 1
s1_table['gee_task_sent'] = ""
# s1_table[s1_table['year'] == 2018]

print("Total number of images:",s1_table.shape[0])
print("Number of selected images:",np.sum(s1_table['sample'].values))

s1_table_path = os.path.join(shp_gdrive_path, 's1_table.csv')

s1_table.to_csv(s1_table_path, index = False)

Total number of images: 575
Number of selected images: 403


In [35]:
im_name

'S1A_IW_GRDH_1SDV_20150702T225157_20150702T225222_006638_008DBE_4A71'

In [141]:
im_name = s1_table.names.values[12]
def check_s1_table_image(s1_table_path, im_name):
    # check if we can download file

    # 1 read s1_table
    s1_table = pd.read_csv(s1_table_path, keep_default_na = False)

    # 2 subset to image
    s1_table_im = s1_table[s1_table['names'] == im_name]
    downloaded = s1_table_im.downloaded.values.item()
    gee_task_sent = s1_table_im.gee_task_sent.values.item()
    # 3 return status as "gee_task_sent", "downloaded", or "not_downloaded"
    if downloaded:
        im_status = 'downloaded'
    elif not gee_task_sent == '':
        im_status = 'gee_task_sent'
    else:
        im_status = 'not_downloaded'

    return im_status

foo = check_s1_table_image(s1_table_path, im_name)
foo

'downloaded'

In [109]:
type(s1_table.gee_task_sent.values[0])

float

In [145]:
# update_s1_table_gee_task_sent

def update_s1_table_gee_task_sent(s1_table_path, im_name, val):
    # update specific image in s1_table with val
    # 1 read s1_table
    s1_table = pd.read_csv(s1_table_path)

    # 2 insert val to talbe
    s1_table.loc[s1_table['names'] == im_name, 'gee_task_sent'] = val
    
    s1_table.to_csv(s1_table_path, index = False)

    return s1_table


s1_table = update_s1_table_gee_task_sent(s1_table_path, im_name, val)

s1_table[s1_table['names'] == im_name]

,names,datestr,date,days_until_next,doy,year,sample,downloaded,gee_task_sent
19,S1A_IW_GRDH_1SDV_20160227T225153_20160227T2252...,20160227,2016-02-27,12.0,58,2016,True,True,2023-01-15


In [143]:
def update_s1_table_all(s1_table_path):
    # check all downloaded files

    # 1 read s1_table
    s1_table = pd.read_csv(s1_table_path, keep_default_na = False)
    
    # 2 check/updated all downloaded files
    images_downloaded = [re.sub("\\.tif$","",x) for x in os.listdir(shp_gee_folder_path)]

    s1_table['downloaded'] = s1_table['names'].isin(images_downloaded)
    s1_table.to_csv(s1_table_path, index = False)
    
    return s1_table

s1_table = update_s1_table_all(s1_table_path)
s1_table

,names,datestr,date,days_until_next,doy,year,sample,downloaded,gee_task_sent
0,S1A_IW_GRDH_1SDV_20150304T225147_20150304T2252...,20150304,2015-03-04,24.0,63,2015,True,True,
1,S1A_IW_GRDH_1SDV_20150328T225148_20150328T2252...,20150328,2015-03-28,24.0,87,2015,True,True,2023-01-15
2,S1A_IW_GRDH_1SDV_20150421T225148_20150421T2252...,20150421,2015-04-21,48.0,111,2015,True,True,2023-01-15
3,S1A_IW_GRDH_1SDV_20150608T225154_20150608T2252...,20150608,2015-06-08,24.0,159,2015,True,True,2023-01-15
4,S1A_IW_GRDH_1SDV_20150702T225157_20150702T2252...,20150702,2015-07-02,48.0,183,2015,True,True,2023-01-15
...,...,...,...,...,...,...,...,...,...
570,S1A_IW_GRDH_1SDV_20221216T225252_20221216T2253...,20221216,2022-12-16,8.0,350,2022,True,False,
571,S1A_IW_GRDH_1SDV_20221224T111302_20221224T1113...,20221224,2022-12-24,4.0,358,2022,True,False,
572,S1A_IW_GRDH_1SDV_20221228T225251_20221228T2253...,20221228,2022-12-28,8.0,362,2022,True,False,
573,S1A_IW_GRDH_1SDV_20230105T111301_20230105T1113...,20230105,2023-01-05,4.0,5,2023,True,False,


In [142]:
def download_shp_s1(s1, shp_name, im_name, ee_geometry, shp_gee_folder_name, s1_table_path):
    
    im_status = check_s1_table_image(s1_table_path, im_name)
    
    if im_status == "not_downloaded":
        s1_im_ic = s1.filterMetadata('system:index','equals',im_name)
        assert s1_im_ic.size().getInfo() == 1
        s1_im = s1_im_ic.first()
        s1_im_shp_name = im_name + '_' + shp_name

        print(f'Sending {shp_name}/{im_name} to GEE')
        task = ee.batch.Export.image.toDrive(
            image = s1_im.float(),
            description = s1_im_shp_name,
            folder = shp_gee_folder_name,
            fileNamePrefix = im_name,
            region = ee_geometry,
            scale = 10,
            maxPixels = 1000000,
            fileFormat = 'GeoTIFF')
        task.start()
        
        val = datetime.strftime(datetime.today(), "%Y-%m-%d")
        update_s1_table_gee_task_sent(s1_table_path, im_name, val)
        
    else:
        print(f'Image {shp_name}/{im_name} skipped, due to status: {im_status}')
    
    return 1


# s1_im.double().getInfo()

In [149]:
for im_name in s1_table.names.values[19:23]:
    download_shp_s1(s1, shp_name, im_name, ee_geometry, shp_gee_folder_name, s1_table_path)

Image SquareBuffer1km/S1A_IW_GRDH_1SDV_20160227T225153_20160227T225218_010138_00EF3C_0DD6 skipped, due to status: downloaded
Image SquareBuffer1km/S1A_IW_GRDH_1SDV_20160310T225153_20160310T225218_010313_00F43F_6E04 skipped, due to status: gee_task_sent
Image SquareBuffer1km/S1A_IW_GRDH_1SDV_20160322T225154_20160322T225219_010488_00F925_04CA skipped, due to status: gee_task_sent
Image SquareBuffer1km/S1A_IW_GRDH_1SDV_20160403T225208_20160403T225233_010663_00FE38_9DC4 skipped, due to status: gee_task_sent


'2023-01-15'

In [150]:
for im_name in s1_table.names.values[1:20]:
    val = datetime.strftime(datetime.today(), "%Y-%m-%d")
    update_s1_table_gee_task_sent(s1_table_path, im_name, val)

In [27]:
ee_ops = ee.data.listOperations()

In [34]:
# import json
# type(ee_ops)

list

In [24]:
im_name = s1_table.names[s1_table['sample']].values[4]
ee_geometry = sq1km_ee.first().geometry()

download_shp_s1(s1, shp_name, im_name, ee_geometry, shp_gee_folder_name, s1_table_path)

Sending SquareBuffer1km/S1A_IW_GRDH_1SDV_20150702T225157_20150702T225222_006638_008DBE_4A71 to GEE


1

In [ ]:
shp_gdrive_path

In [ ]:

# download_path = os.path.join(google_drive_path, 's1_tiles_download')
# def check_s1_image_status(shp_gdrive_path, shp_gee_folder_name):
s1_table = pd.read_csv(s1_table_path)
images_downloaded = [re.sub("\\.tif$","",x) for x in os.listdir(shp_gee_folder_path)]

s1_table['downloaded'] = s1_table['names'].isin(images_downloaded)
# s1_table['downloaded']
s1_table

In [ ]:
ee.data.listOperations()

In [ ]:
# resample_names = []
resample_names.append([1])
resample_names

In [ ]:
s1_table[s1_table['days_until_next'] < 0]
# s1_table.loc[395:397]

In [ ]:
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format

(ggplot() +       # new
 geom_point(data = s1_table, mapping = aes('date', 'days_until_next', color = 'sample', shape = 'sample')) + 
 scale_shape_manual(values = ["|", "+"]) +
 scale_x_datetime(breaks=date_breaks('2 years'), labels = date_format("%Y %b")))


In [ ]:


# def DownloadS2pt(sample_pt_xy, loc_id, timeseries_dir_path, date_range):
    
sample_pt_name = 'pt_ts_' + str(loc_id)

sample_pt = ee.Geometry.Point(sample_pt_xy)

timeseries_dir_name = Path(timeseries_dir_path).parts[-1]
s2_colname = 'pt_ts_loc_s2'
s2_pt_filename = re.sub('loc_', 'loc_' + str(loc_id) +'_', s2_colname) #sample_pt_name + '_s2'
s2_pt_filepath = os.path.join(timeseries_dir_path, s2_pt_filename + '.csv')

# if os.path.exists(s2_pt_filepath):
#     dummy = s2_pt_filepath
#     print(s2_pt_filename + '.csv already exists')

# else:

s2_output_bands = ['B8','B4','B3','B2','clouds','cloudmask','shadows','probability']

# params variable is used to pass  information to the cloud masking functions.
# see help(add_cld_shadow_mask_func)
s2params = {
    'START_DATE' : date_range[0],
    'END_DATE' : date_range[1],
    'CLOUD_FILTER' : 50,
    'CLD_PRB_THRESH' : 53, # 53 for Cauvery # 55 for Indus
    'NIR_DRK_THRESH' : 0.2,
    'CLD_PRJ_DIST' : 1,
    'BUFFER' : 50
}

s2_clouds_ic = ees.get_s2_sr_cld_col(sample_pt, s2params) \
  .map(ees.add_cld_shadow_mask_func(s2params))

# For some reason the reproject() works so that subsequent sampling returns the whole rectangular array
# see https://stackoverflow.com/questions/64012752/gee-samplerectangle-returning-1x1-array
# s2_clouds_im = s2_clouds_ic.mosaic().reproject(crs = ee.Projection('EPSG:4326'), scale=10) #.clip(hyd_watershed)

# Get pixel timeseries
s2_ts = rs.get_pixel_ts_allbands(
    pts_fc = ee.FeatureCollection(sample_pt),
    image_collection = s2_clouds_ic,
    ic_property_id = 'system:index',
    scale = 10) # for Landsat resolution
# time_series_pd_load = geemap.ee_to_pandas(time_series_fc)

task_s2 = ee.batch.Export.table.toDrive(
    collection = s2_ts,
    selectors = s2_output_bands + ['image_id'],
    folder = timeseries_dir_name,
    description = s2_pt_filename,
    fileNamePrefix = s2_pt_filename)

task_s2.start()


print('Generating ' + s2_pt_filename + '.csv')

## Function to download sentinel 2 and Landsat 8 points

In [ ]:
def DownloadS2pt(sample_pt_xy, loc_id, timeseries_dir_path, date_range):
    
    sample_pt_name = 'pt_ts_' + str(loc_id)
    
    sample_pt = ee.Geometry.Point(sample_pt_xy)
    
    timeseries_dir_name = Path(timeseries_dir_path).parts[-1]
    s2_colname = 'pt_ts_loc_s2'
    s2_pt_filename = re.sub('loc_', 'loc_' + str(loc_id) +'_', s2_colname) #sample_pt_name + '_s2'
    s2_pt_filepath = os.path.join(timeseries_dir_path, s2_pt_filename + '.csv')
    
    if os.path.exists(s2_pt_filepath):
        dummy = s2_pt_filepath
        print(s2_pt_filename + '.csv already exists')
        
    else:
    
        s2_output_bands = ['B8','B4','B3','B2','clouds','cloudmask','shadows','probability']
        
        # params variable is used to pass  information to the cloud masking functions.
        # see help(add_cld_shadow_mask_func)
        s2params = {
            'START_DATE' : date_range[0],
            'END_DATE' : date_range[1],
            'CLOUD_FILTER' : 50,
            'CLD_PRB_THRESH' : 53, # 53 for Cauvery # 55 for Indus
            'NIR_DRK_THRESH' : 0.2,
            'CLD_PRJ_DIST' : 1,
            'BUFFER' : 50
        }
        
        s2_clouds_ic = ees.get_s2_sr_cld_col(sample_pt, s2params) \
          .map(ees.add_cld_shadow_mask_func(s2params))
        
        # For some reason the reproject() works so that subsequent sampling returns the whole rectangular array
        # see https://stackoverflow.com/questions/64012752/gee-samplerectangle-returning-1x1-array
        # s2_clouds_im = s2_clouds_ic.mosaic().reproject(crs = ee.Projection('EPSG:4326'), scale=10) #.clip(hyd_watershed)
        
        # Get pixel timeseries
        s2_ts = rs.get_pixel_ts_allbands(
            pts_fc = ee.FeatureCollection(sample_pt),
            image_collection = s2_clouds_ic,
            ic_property_id = 'system:index',
            scale = 10) # for Landsat resolution
        # time_series_pd_load = geemap.ee_to_pandas(time_series_fc)
            
        task_s2 = ee.batch.Export.table.toDrive(
            collection = s2_ts,
            selectors = s2_output_bands + ['image_id'],
            folder = timeseries_dir_name,
            description = s2_pt_filename,
            fileNamePrefix = s2_pt_filename)
        
        task_s2.start()
        
        
        print('Generating ' + s2_pt_filename + '.csv')
        
def DownloadOLI8pt(sample_pt_xy, loc_id, timeseries_dir_path, date_range):
    
    sample_pt_name = 'pt_ts_' + str(loc_id)
    
    sample_pt = ee.Geometry.Point(sample_pt_xy)
    
    timeseries_dir_name = Path(timeseries_dir_path).parts[-1]
    oli8_colname = 'pt_ts_loc_oli8'
    oli8_pt_filename = re.sub('loc_', 'loc_' + str(loc_id) +'_', oli8_colname) #sample_pt_name + '_oli8'
    oli8_pt_filepath = os.path.join(timeseries_dir_path, oli8_pt_filename + '.csv')
    
    if os.path.exists(oli8_pt_filepath):
        print(oli8_pt_filepath + ' already exists')

    else:
        
        oli8_output_bands = ['SR_B7','SR_B6','SR_B5','SR_B4','SR_B3','SR_B2','clouds','shadows','clouds_shadows']
        oli8_ic = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
          .filterBounds(sample_pt) \
          .filterDate(date_range[0],date_range[1])
          
        get_qaband_clouds_shadows = rs.get_qaband_clouds_shadows_func(
              qa_bandname = 'QA_PIXEL', 
              cloud_bit = 3, 
              shadow_bit = 4,
              keep_orig_bands = True) 
        oli8_clouds_ic = (oli8_ic
          .map(get_qaband_clouds_shadows))
          # .map(lambda im: im.addBands(im.expression('im.clouds | im.clouds_shadows', {'im' : im}).rename('cloudmask'))))
          
        # Get oli8 pixel timeseries
        oli8_ts = rs.get_pixel_ts_allbands(
            pts_fc = ee.FeatureCollection(sample_pt),
            image_collection = oli8_clouds_ic,
            ic_property_id = 'system:index',
            scale = 30) # for Landsat resolution
        # time_series_pd_load = geemap.ee_to_pandas(time_series_fc)
    
        # oli8_output_bands = ['B8','B4','B3','B2','clouds','cloudmask','shadows','probability']
        
        
        # For some reason the reproject() works so that subsequent sampling returns the whole rectangular array
        # see https://stackoverflow.com/questions/64012752/gee-samplerectangle-returning-1x1-array
        # oli8_clouds_im = oli8_clouds_ic.mosaic().reproject(crs = ee.Projection('EPSG:4326'), scale=10) #.clip(hyd_watershed)
        
            
        task_oli8 = ee.batch.Export.table.toDrive(
            collection = oli8_ts,
            selectors = oli8_output_bands + ['image_id'],
            folder = timeseries_dir_name,
            description = oli8_pt_filename,
            fileNamePrefix = oli8_pt_filename)
        
        task_oli8.start()
                
        # print('Generating ' + oli8_pt_filename + '.csv')
        print('Generating ' + oli8_pt_filename + '.csv')

In [ ]:
sys.path.append('/Users/gopal/Projects/ml/manclassify')
from geemodules import rs
from geemodules import eesentinel as ees
from pathlib import Path
import os

In [ ]:
ee.Initialize()

In [ ]:
timeseries_dir_path = '/Users/gopal/Google Drive/_Research/Research projects/G_D/survey_pts/mig_survey_GEE'
date_range = ['2012-06-01','2022-06-01']

num_pts = survey_pts.shape[0]

for i in np.arange(1062, num_pts):
    sample_pt_xy = [survey_pts.lon.iloc[i],survey_pts.lat.iloc[i]]
    loc_id = survey_pts.fid.iloc[i]
    # print(f'{i+1} of {num_pts} pts, location id: {loc_id}')
    # DownloadOLI8pt(sample_pt_xy, loc_id, timeseries_dir_path, date_range)
    # DownloadS2pt(sample_pt_xy, loc_id, timeseries_dir_path, date_range)

In [ ]:
my_new_string = re.sub('[\s\n\.]', '', loc_id)
# loc_id
my_new_string

In [ ]:
print(f'{1+1} of 4')
1